In [2]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model
import nltk

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [60]:
# Hyperparameters
MAX_DOCUMENT_LENGTH = 20
VOCAB_SIZE = 10000
DATA_TEST_SIZE = 100

# Model Parameters
RNN_CELL_SIZE = 128
EMBEDDING_DIM = 128

# Training Parameters
TRAIN_EPOCHS = 10
TRAIN_BATCH_SIZE = 8
TRAIN_EVAL_EVERY = 1000

In [61]:
ds = data.create_reddit_inputs("./data", batch_size=TRAIN_BATCH_SIZE, train_epochs=TRAIN_EPOCHS)

In [124]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(RNN_CELL_SIZE)
    # sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.dynamic_rnn(cell, sequences, dtype=tf.float32, sequence_length=sequence_lengths)

In [135]:
# Monitor for sampling sentences from the estimator
sample_mon = model.SentenceSampleMonitor(
    vocab=ds.vocab,
    max_sample_length=10,
    num_samples=5,    
    every_n_steps=TRAIN_EVAL_EVERY,
    first_n_steps=-1)

# Monitor for development set loss
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=ds.dev_input_fn,
    every_n_steps=TRAIN_EVAL_EVERY)

# The function to create the model
model_fn = model.create_language_model_rnn(
    vocab_size=len(ds.vocab),
    embedding_dim=EMBEDDING_DIM,
    rnn_fn=simple_rnn,
    max_sequence_len=MAX_DOCUMENT_LENGTH)

model_name = "maxlen_{}_rnn{}_embed_{}".format(MAX_DOCUMENT_LENGTH, RNN_CELL_SIZE, EMBEDDING_DIM)
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir="./checkpoints/{}".format(model_name))
estimator.fit(input_fn=ds.train_input_fn, steps=None, monitors=[sample_mon, dev_monitor])

Tensor("loss/losses/TensorArrayPack:0", shape=(?, 8), dtype=float32)
Tensor("loss/losses/TensorArrayPack:0", shape=(?, 8), dtype=float32)
Tensor("loss/truediv:0", shape=(), dtype=float32)


INFO:tensorflow:Create CheckpointSaver


KeyboardInterrupt: 

ValueError: Shape (8, ?, 128) must have rank 1